# Introduction
This 

In [1]:
import numpy as np


In [3]:
class Network:
    
    def __init__(self, architecture, activation= 'sigmoid'):
        self.architecture= architecture
        self.activation= activation
        self.parameters=[]
        
        if type(self.activation) != str:
            assert len(self.activation) + 1 == len(self.architecture)
        for i in range(len(architecture) -1):
            layer_parameters= self.create_weights_and_biases(architecture[i], architecture[i + 1])
            self.parameters.append(layer_parameters)
        
        
    def create_weights_and_biases( in_nodes, out_nodes):
        std= np.sqrt(2.0/(in_nodes + out_nodes))
        w= np.random.normal(loc= 0, scale= std, size= (in_nodes, out_nodes))
        b= np.zeros((1, out_nodes))
        return {'weight':w , 'bias':b}